In [2]:
import lmstudio as lms

model_name="phi-4" # "llama-3.2-1b-instruct"
model = lms.llm(model_name)

In [29]:
query = """
query: rephrase the question as a 10 year old kid who has to use few words - so eliminate any unnecessary words but not proper nouns. Don't answer the question. Reply in json with no markup - with only one tag called 'rephrased_question'. Follow the examples:
Input: "What happened at Starrcade '88 involving Rhodes and Sting?" output: "What happened with Rhodes and Sting at Starrcade '88?"
Input: "What skills do graduates of the jewelry making and repair course acquire?" output: "What do graduates of the jewelry and repair course learn?"
Input: "What role do marine microorganisms play in mangrove ecosystems?" output: "What role do microorganisms play in mangroves?"
Input: "What was the public reaction to the proposed railway changes in the Beeching Report?" "What was the popular reaction to the changes in the Beeching Report?"
Input: """
def getq(question):
    return f"{query}{question}"

In [31]:
questions = [
    "What is the meaning of life?",
    "Which skills do graduates of the jewelry making and repair course acquire?",
    "Why was Habib Thameur chosen as head of the Maghrebi office?"
]
# result = model.respond(getq(''))
for q in questions:
    result = model.respond(getq(f'"{q}'))
    print(result)

```json
{
  "rephrased_question": "What's the meaning of life?"
}
```
```json
{
  "rephrased_question": "What skills do graduates of the jewelry and repair course learn?"
}
```
```json
{
  "rephrased_question": "Why did they pick Habib Thameur for the Maghrebi office?"
}
```


In [32]:
data = []
import json
with open("../benchmark/mldr/may5_8_20_progress_1.fixed.jsonl") as inp:
    for line in inp:
        data.append(json.loads(line))

In [33]:
json.loads(result.content.replace('```json\n','').replace('\n```',''))

{'rephrased_question': 'Why did they pick Habib Thameur for the Maghrebi office?'}

In [35]:
from tqdm.notebook import tqdm
from docuverse.utils import get_param
for d in tqdm(data):
    ans = model.respond(getq(f'"{d["question"]}"'))
    try:
        res = json.loads(ans.content.replace('```json\n','').replace('\n```',''))
        d['rephrased_question'] = get_param(res, 'rephrased_question|output', "")
    except Exception as e:
        print(f"Failed the question '{d['question']}' with the output '{ans.content}': e")

  0%|          | 0/19899 [00:00<?, ?it/s]

Failed the question 'Why did Mujib seek recognition for the term "Bangla-desh"?' with the output '```json
{
  "rephrased_question": "Why did Mujib want "Bangla-desh" recognized?"
}
```': e
Failed the question 'What special item is unlocked by achieving "S" rank in all 35 bonus missions?' with the output '```json
{
  "rephrased_question": "What special thing do you get for "S" rank in all 35 bonus missions?"
}
```': e
Failed the question 'What was the purpose behind the "CyClone Dairy" hoax?' with the output '```json
{
  "rephrased_question": "Why was there a "CyClone Dairy" trick?"
}
```': e
Failed the question 'What was the rationale behind diagnosing political dissidents with "sluggish schizophrenia" in the USSR?' with the output '```json
{
  "rephrased_question": "Why did they call political opponents "sluggish schizophrenia" in the USSR?"
}
```': e
Failed the question 'What is the significance of the "Train leaving on track five" gag in the context of Jack Benny's radio shows?' wit

In [46]:
with open("../benchmark/mldr/may5_8_20_progress_1.fixed.rephrase.jsonl", "w") as out:
    for d in data:
        out.write(json.dumps(d)+"\n")

In [38]:
ans=model.respond(getq("What special item is unlocked by achieving 'S' rank in all 35 bonus missions?"))
json.loads(ans.content.replace('```json\n','').replace('\n```',''))

{'rephrased_question': 'What special thing do you get for top scores on all 35 extra games?'}

In [43]:
for d in data:
    if 'rephrased_question' not in d:
        question = d['question'].replace('"', "'")
        print(question)
        ans = model.respond(getq(f'"{question}"'))
        try:
            res = json.loads(ans.content.replace('```json\n','').replace('\n```',''))
            d['rephrased_question'] = get_param(res, 'rephrased_question|output', "")
            print(f"out: {d['rephased_question']}")
        except Exception as e:
            print(f"Failed the question '{d['question']}' with the output '{ans.content}': e")


In [45]:
[d['rephrased_question'] for d in data if d['question'].find("Train leaving")>=0]

["Why is the 'Train leaving on track five' joke important for Jack Benny's radio shows?"]